In [1]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
matplotlib.style.use('ggplot')

import requests
import json
from ast import literal_eval

In [2]:
panoptes_comments = pd.read_csv('/srv/zooniverse/raw_data/panoptes/panoptes_comments_2017-03-21.csv')

In [3]:
panoptes_comments = panoptes_comments.rename(columns={'id':'comment_id',
                                                      'focus_type':'focus_base_type',
                                                      'user_login':'user_name',
                                                      'mentioning':'mentions',
                                                      'tagging':'tags',
                                                      'reply_id':'response_to_id',
                                                      'project_id':'panoptes_project_id'})
panoptes_comments['panoptes_dump'] = 1

In [4]:
panoptes_comments = panoptes_comments[['comment_id','body','focus_id','focus_base_type','discussion_id','user_id','user_name','created_at','mentions','tags','panoptes_project_id','response_to_id','board_id','panoptes_dump']]

In [5]:
ouroboros_comments = pd.read_csv('/srv/zooniverse/tables/ouroboros_discussion_table_100000.csv')

In [6]:
# create temp series with comments list expanded into individual rows, indexed by discussion
ouroboros_comments_temp = ouroboros_comments.apply(lambda x: pd.Series(json.loads(x['comments'])),axis=1).stack().reset_index(level=1, drop=True)
# rename the series for merging
ouroboros_comments_temp.name = 'comment'
# merge with discussion df
ouroboros_comments = ouroboros_comments.drop('comments',axis=1).join(ouroboros_comments_temp)
# rename fields
ouroboros_comments = ouroboros_comments.rename(columns={'_id':'discussion_id',
                                                        'focus':'discussion_focus',
                                                        'project_id':'ouroboros_mongo_id'})
# reset the index for expantion of discussion column
ouroboros_comments = ouroboros_comments.reset_index(drop=True)
# drop meta columns
ouroboros_comments = ouroboros_comments[['board','discussion_id','discussion_focus','ouroboros_mongo_id','comment']]

In [7]:
# expand the discussion column into multiple rows
ouroboros_comments_temp = ouroboros_comments['comment'].apply(pd.Series)


/home/sjm668/dev/zooniverse/src/venv/lib/python3.5/site-packages/pandas/indexes/api.py:37: RuntimeWarning: unorderable types: int() > str(), sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/home/sjm668/dev/zooniverse/src/venv/lib/python3.5/site-packages/pandas/indexes/api.py:37: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/home/sjm668/dev/zooniverse/src/venv/lib/python3.5/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: str() > int(), sort order is undefined for incomparable objects
  result = result.union(other)


In [18]:
# expand the focus column into multiple rows
ouroboros_comments_temp = ouroboros_comments['discussion_focus'].apply(lambda x: pd.Series(json.loads(x)))
# rename the _id field and base_type fields
ouroboros_comments_temp = ouroboros_comments_temp.rename(columns={'_id':'focus_id','base_type':'focus_base_type'})
ouroboros_comments_temp = ouroboros_comments_temp[['focus_id','focus_base_type']]
# merge discussion meta data with expanded comment df
ouroboros_comments = ouroboros_comments.merge(ouroboros_comments_temp,right_index=True,left_index=True)
# drop old comment field
ouroboros_comments = ouroboros_comments.drop('discussion_focus',axis=1)

In [19]:
# remove dict keys for dicts stored as string
def expand_text_field(text_field):
    return text_field.split(':')[1].split("'")[1]

# remove dict keys for dicts stored as dict, add ObjectId text for consistancy w/ mongo
def expand_oid_dict_field(dict_field):
    try:
        dict_field = literal_eval(dict_field)
        if '_id' in dict_field:
            return 'ObjectId({0})'.format(dict_field['_id'])
    except ValueError:
        return None
    except TypeError:
        return None

# remove dict keys for dicts stored as dict,    
def expand_datetime_dict_field(dict_field):
    if type(dict_field) == dict:
        return next(iter(dict_field.values()))
    else:
        return None
    

In [20]:
# remove dictionary formating from comment_id
ouroboros_comments['comment_id'] = ouroboros_comments['comment_id'].apply(expand_oid_dict_field)
# remove dictionary formatting from created_at
ouroboros_comments['created_at'] = ouroboros_comments['created_at'].apply(expand_datetime_dict_field)
# convert created_at to datetime object
ouroboros_comments['created_at'] = pd.to_datetime(ouroboros_comments['created_at'])
# remove dictionary formatting from user_id
ouroboros_comments['user_id'] = ouroboros_comments['user_id'].apply(expand_oid_dict_field)
# get id field from board column
ouroboros_comments['board_id'] = ouroboros_comments['board'].apply(expand_oid_dict_field)

In [21]:
# load project csv
project_df = pd.read_csv('/srv/zooniverse/tables/all_projects_table_05-31-17.csv')
# merge on ouroboros_mongo_id field to include panoptes id
ouroboros_comments = ouroboros_comments.merge(project_df[['ouroboros_mongo_id','panoptes_project_id']],on='ouroboros_mongo_id')

In [38]:
# add column for ouroboros info
ouroboros_comments['ouroboros_dump'] = 1
# drop columns
result_df = ouroboros_comments[['board_id','discussion_id','focus_id','focus_base_type','ouroboros_mongo_id','comment_id','body','created_at','mentions','response_to','response_to_id','tags','upvotes','user_id','user_name','user_zooniverse_id','panoptes_project_id','ouroboros_dump']]

In [39]:
result_df = result_df.append(panoptes_comments)

In [49]:
# mark all classifications in the panoptes API == 1
result_df.loc[result_df['panoptes_project_id'].isin(project_df.loc[project_df['panoptes_api'] == 1]['panoptes_project_id']),'panoptes_api'] = 1
result_df.loc[result_df['panoptes_project_id'].isin(project_df.loc[project_df['panoptes_api'] == 0]['panoptes_project_id']),'panoptes_api'] = 0
result_df.loc[result_df['panoptes_dump'].isnull(),'panoptes_dump'] = 0
result_df.loc[result_df['ouroboros_dump'].isnull(),'ouroboros_dump'] = 0

In [58]:
def mark_project_df(project_df,result_df):
    project_df.loc[project_df['panoptes_project_id'].isin(result_df['panoptes_project_id']), 'talk'] = 1
    project_df.loc[project_df['talk'].isnull(), 'talk'] = 0
    return project_df

In [60]:
mark_project_df(project_df,result_df)['talk']

0      1.0
1      1.0
2      0.0
3      1.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      1.0
10     0.0
11     1.0
12     0.0
13     1.0
14     0.0
15     1.0
16     0.0
17     1.0
18     0.0
19     0.0
20     1.0
21     1.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     1.0
28     1.0
29     1.0
      ... 
863    0.0
864    0.0
865    1.0
866    0.0
867    0.0
868    0.0
869    0.0
870    0.0
871    0.0
872    0.0
873    0.0
874    0.0
875    0.0
876    0.0
877    0.0
878    0.0
879    0.0
880    0.0
881    0.0
882    0.0
883    0.0
884    0.0
885    0.0
886    0.0
887    0.0
888    0.0
889    0.0
890    0.0
891    0.0
892    0.0
Name: talk, dtype: float64